In [ ]:
import random
import numpy as np
import pandas as pd
from collections import Counter
from copy import deepcopy
import sys

sys.path.append('../protosp03/recommendation/')
sys.path.append('../protosp03/data/synthetic/')
import matchings
import market
import upskillings
import make_synthetic

In [ ]:
path = "../data/taxonomy/taxonomy_V4.csv"
mastery_levels = [1, 2, 3, 4]
years = [i for i in range(2023, 2017, -1)]

In [ ]:
skills, learners, jobs, courses = make_synthetic.get_job_market(path, mastery_levels, years )

In [ ]:
skill_supply, skill_demand, skill_trends, skills_attractiveness = market.get_all_market_metrics(skills, mastery_levels, learners, jobs, years)

In [ ]:
applicability_threshold = 40
up_skilling_advice = upskillings.up_skilling_advice(learners[0], jobs[0], skills_attractiveness)
print("Current number of jobs that can be applied to:", matchings.get_nb_applicable_jobs(learners[0], jobs, applicability_threshold))
print("Number of new jobs that can be applied to after up_skilling:", matchings.get_increased_nb_applicable_jobs(learners[0], jobs, up_skilling_advice, applicability_threshold))

In [ ]:
enrollable_courses = matchings.get_all_enrollable_courses(learners[0], courses, threshold=1.0)

In [ ]:
enrollable_courses

In [ ]:
for course in enrollable_courses:
    print(matchings.learner_course_provided_matching(learners[0], course), course["provided_skills"])


In [ ]:
test